<a href="https://colab.research.google.com/github/shanvelc/module4/blob/main/Shan_M4_NB_MiniProject_01_MLR_MPI_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Implementation of Multiple Linear Regression using MPI and OpenMP

## Learning Objectives

At the end of the mini-project, you will be able to :

* understand the collective communication operations like scatter, gather, broadcast
* understand the blocking and non-blocking communication
* implement multiple linear regression and run it using MPI
* implement the multiple linear regression based predictions using OpenMP

### Dataset

The dataset chosen for this mini-project is [Combined Cycle Power Plant](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant). The dataset is made up of 9568 records and 5 columns. Each record contains the values for Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output.

Predicting full load electrical power output of a base load power plant is important in order to maximize the profit from the available megawatt hours.  The base load operation of a power plant is influenced by four main parameters, which are used as input variables in the dataset, such as ambient temperature, atmospheric pressure, relative humidity, and exhaust steam pressure. These parameters affect electrical power output, which is considered as the target variable.

**Note:** The data was collected over a six year period (2006-11).

## Information

#### MPI in a Nutshell

MPI stands for "Message Passing Interface". It is a library of functions (in C / Python) or subroutines (in Fortran) that you insert into source code to perform data communication between processes. MPI was developed over two years of discussions led by the MPI Forum, a group of roughly sixty people representing some forty organizations.

To know more about MPI click [here](https://hpc-tutorials.llnl.gov/mpi/)


#### Multiple Linear Regression

Multiple regression is an extension of simple linear regression. It is used when we want to predict the value of a variable based on the value of two or more other variables. The variable we want to predict is called the dependent variable (or sometimes, the outcome, target or criterion variable). The variables we are using to predict the value of the dependent variable are called the independent variables (or sometimes, the predictor, explanatory or regressor variables).

**Note:** We will be using the mpi4py Python package for MPI based code implementation

## Grading = 20 Points

**Run the below code to install mpi4py package**

#### Importing Necessary Packages

In [ ]:
# Importing pandas
import pandas as pd
# Importing Numpy
import numpy as np
# Importing sqrt function from the Math
from math import sqrt
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP
import time

#### Downloading the data

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv

### Overview

* Load the data and perform data pre-processing
* Identify the features, target and split the data into train and test
* Implement multiple Linear Regression by estimating the coefficients on the given data
* Use MPI package to distribute the data and implement `communicator`
* Define functions for each objective and make a script (.py) file to execute using MPI command
* Use OpenMP component to predict the data and calculate the error on the predicted data
* Implement the Linear Regression from `sklearn` and compare the results

#### Exercise 1: Load data (1 point)

Write a function that takes the filename as input and loads the data in a pandas dataframe with the column names as Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output respectively.

**Hint:** read_csv()


In [ ]:
FILENAME = "/content/PowerPlantData.csv" # File path

# YOUR CODE HERE to Define a function to load the data
def loadData(filename):
  return pd.read_csv(filename, names= ['AmbientTemperature', 'ExhaustVaccum', 'AmbientPressure', 'RelativeHumidity', 'EnergyOutput'], header=0)

In [ ]:
df = loadData(FILENAME)
df.head()

#### Exercise 2: Explore data (1 point)

Write a function that takes the data loaded using the above defined function as input and explore it.

**Hint:** You can define and check for following things in the dataset inside a function

- checking for the number of rows and columns
- summary of the dataset
- check for the null values
- check for the duplicate values

In [ ]:
# YOUR CODE HERE
def exploreData(df):
  print('Number of rows: {}'.format(df.shape[0]))
  print('Number of columns: {}'.format(df.shape[1]))
  print('Dataset Summary', df.describe())
  print('Null Values', df.isnull().sum())
  print('Duplicate Values', df.duplicated().sum())
  return None

In [ ]:
exploreData(df)

#### Exercise 3: Handle missing data (1 point)

After exploring the dataset if there are any null values present in the dataset then define a function that takes data loaded using the above defined function as input and handle the null values accordingly.

**Hint:**

- Drop the records containing the null values - dropna()
- Replace the null values with the mean/median/mode - fillna()

In [ ]:
# Function to handle missing data

# YOUR CODE HERE
def handleMissingData(df):
  df.dropna(inplace=True)
  df.fillna(df.mean(), inplace=True)
  return df

In [ ]:
df = handleMissingData(df)
exploreData(df)

#### Exercise 4: Scale the data (1 point)

Write a function that takes the data after handling the missing data as input and returns the standardized data.

**Hint:**

- standardization of the data  can be performed using the below formula

$ (x - mean(x)) / std(x) $

In [ ]:
# Defining a function to standardize the data

# YOUR CODE HERE
def scaleData(df):
  return (df - df.mean())/df.std()

In [ ]:
df = scaleData(df)
df.head()

#### Exercise 5: Feature selection (1 point)

Write a function that takes scaled data as input and returns the features and target variable values

**Hint:**

- Features: AmbientTemperature, ExhaustVaccum, AmbientPressure, RelativeHumidity
- Target Variable: EnergyOutput

In [ ]:
# Define a function

# YOUR CODE HERE
def featureTargetSplit(df):
  features = df.drop('EnergyOutput', axis=1)
  target = df['EnergyOutput']
  return features, target

In [ ]:
features, target = featureTargetSplit(df)
print(features.head())
print(target.head())

#### Exercise 6: Correlation (1 point)

Calculate correlation between the variables

In [ ]:
# YOUR CODE HERE
#df.corr().style.background_gradient(cmap='coolwarm')

import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(df.corr(), cmap='RdBu')
plt.show()

#### Exercise 7: Estimate the coefficients (2 points)

Write a function that takes features and target as input and returns the estimated coefficient values

**Hint:**

- Calculate the estimated coefficients using the below formula

$ β = (X^T X)^{-1} X^T y $

- transpose(), np.linalg.inv()

In [ ]:
# Calculating the coeffients

# YOUR CODE HERE
def estimated_coefficients(x, y):
  return np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)

In [ ]:
coefficients = estimated_coefficients(features, target)
print(coefficients)

#### Exercise 8: Fit the data to estimate the coefficients (2 points)

Write a function named fit which takes features and targets as input and returns the intercept and coefficient values.

**Hint:**

- create a dummy column in the features dataframe which is made up of all ones
- convert the features dataframe into numpy array
- call the estimated coefficients function which is defined above
- np.ones(), np.concatenate()

In [ ]:
# defining a fit function
def fit(x, y):
    # YOUR CODE HERE
    x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
    return estimated_coefficients(x, y)

In [ ]:
coefficients = fit(features, target)
print(coefficients)

#### Exercise 9: Predict the data on estimated coefficients (1 point)

Write a function named predict which takes features, intercept and coefficient values as input and returns the predicted values.

**Hint:**

- Fit the intercept, coefficients values in the below equation

  $y = b_0 + b_1*x + ... + b_i*x_i$

In [ ]:
 # fucntion to predict the values
def predict(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    #YOUR CODE HERE
    predictions = intercept + np.dot(x, coefficients)

    return predictions

In [ ]:
predictions = predict(features, coefficients[0], coefficients[1:])
print(predictions)

#### Exercise 10: Root mean squared error (1 point)

Write a function to calculate the RMSE error.

**Hint:**

- [How to calculate the RSME error](https://towardsdatascience.com/what-does-rmse-really-mean-806b65f2e48e)

In [ ]:
# Define a function to calculate the error
from sklearn.metrics import mean_squared_error
# YOUR CODE HERE
def rmse(actual, predicted):
  return sqrt(mean_squared_error(actual, predicted))

In [ ]:
error = rmse(target, predictions)
print(error)

#### Exercise 11: Split the data into train and test (1 point)

Write a function named train_test_split which takes features and targets as input and returns the train and test sets respectively.

**Hint:**

- Shuffle the data
- Consider 70 % of data as a train set and the rest of the data as a test set

In [ ]:
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
def train_testsplit(x, y):
  return train_test_split(x, y, test_size=0.3, shuffle=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
import multiprocessing

cpuCount = multiprocessing.cpu_count()
print(f"--- Current CPU has {cpuCount} cores ---")

In [ ]:
coefficients_train = fit(x_train, y_train)
#print(coefficients_train)

start_time = time.perf_counter()
predictions_test = predict(x_test, coefficients_train[0], coefficients_train[1:])
#print(predictions_train)

error_train = rmse(y_test, predictions_test)
print(error_train)
print(f"--- Time taken {time.perf_counter() - start_time} seconds with 'Sequential' execution on {cpuCount} cores ---")

#### Exercise 12: Implement predict using OpenMP (1 point)

Get the predictions for test data and calculate the test error(RMSE) by implementing the OpenMP (pymp)

**Hints:**

* Using the pymp.Parallel implement the predict function (use from above)

* Call the predict function by passing test data as an argument

* calculate the error (RMSE) by comparing the Actual test data and predicted test data

In [ ]:
!pip install pymp-pypi

In [ ]:
import pymp
# YOUR CODE HERE
start_time = time.perf_counter()
#cpuCount = 4
with pymp.Parallel(cpuCount) as p:
    predictions_test = predict(x_test, coefficients_train[0], coefficients_train[1:])
    error_test = rmse(y_test, predictions_test)
    print(error_test)
print(f"--- Time taken {time.perf_counter() - start_time} seconds during 'Parallel' execution on {cpuCount} cores ---")

#### Exercise 13: Create a communicator (1 point)

Create a comunicator and define the rank and size

In [ ]:
!pip install mpi4py

In [ ]:
# YOUR CODE HERE

# Importing MPI from mpi4py package
#%%writefile rank.py
from mpi4py import MPI # Importing mpi4py package from MPI module
# Define a function

# creating the communicator
comm = MPI.COMM_WORLD
# number of the process running the code i.e rank
rank = comm.Get_rank()
# total number of processes running i.e size
size = comm.Get_size()
# Displaying the rank and size of a communicator
print("rank is {} and size is {}".format(rank,size))

#### Exercise 14: Divide the data into slices (1 point)

Write a function named dividing_data which takes train features set, train target set, and size of workers as inputs and returns the sliced data for each worker.

![img](https://cdn.iisc.talentsprint.com/CDS/Images/MiniProject_MPI_DataSlice.JPG)

For Example, if there are 4 processes, slice the data into 4 equal parts with 25% ratio

**Hint:**

- Divide the Data equally among the workers
  - Create an empty list
  - Iterate over the size of workers
  - Append each slice of data to the list

In [ ]:
def dividing_data(x_train, y_train, size_of_workers):
    # Size of the slice
    slice_for_each_worker = int(Decimal(x_train.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))
    print('Total train size: {} sliced in {} workers'.format(x_train.shape[0], size_of_workers))
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    # YOUR CODE HERE
    slicedFeatures = []
    slicedTarget = []

    for i in range(size_of_workers):
      slicedFeatures.append(x_train[i*slice_for_each_worker:(i+1)*slice_for_each_worker])
      slicedTarget.append(y_train[i*slice_for_each_worker:(i+1)*slice_for_each_worker])

    return slicedFeatures, slicedTarget


In [ ]:
#cpuCount = 4
slicedDataList = dividing_data(x_train, y_train, cpuCount)
print(slicedDataList)

#### Exercise 15: Prepare the data in root worker to assign data for all the workers (1 point)

- When it is the root worker, perform the below operation:
    - Store the features and target values in separate variables
    - Split the data into train and test sets using the train_test_split function defined above
    - Divide the data among the workers using the dividing_data function above

In [ ]:
print(rank)

In [ ]:
%%writefile mpicode.py

# YOUR CODE HERE to Define a function to load the data
def loadData(filename):
  return pd.read_csv(filename, names= ['AmbientTemperature', 'ExhaustVaccum', 'AmbientPressure', 'RelativeHumidity', 'EnergyOutput'], header=0)

# YOUR CODE HERE
def exploreData(df):
  print('Number of rows: {}'.format(df.shape[0]))
  print('Number of columns: {}'.format(df.shape[1]))
  print('Dataset Summary', df.describe())
  print('Null Values', df.isnull().sum())
  print('Duplicate Values', df.duplicated().sum())
  return None

# YOUR CODE HERE
def handleMissingData(df):
  df.dropna(inplace=True)
  df.fillna(df.mean(), inplace=True)
  print('Missing Data Handled')
  return df

def scaleData(df):
  print('DF Scaled')
  return (df - df.mean())/df.std()

def featureTargetSplit(df):
  features = df.drop('EnergyOutput', axis=1)
  target = df['EnergyOutput']
  return features, target

def estimated_coefficients(x, y):
  return np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)

def fit(x, y):
  # YOUR CODE HERE
  x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
  return estimated_coefficients(x, y)

def predict(x, intercept, coefficients):
	'''
	y = b_0 + b_1*x + ... + b_i*x_i
	'''
	#YOUR CODE HERE
	predictions = intercept + np.dot(x, coefficients)
	return predictions

# Define a function to calculate the error
# YOUR CODE HERE
def rmse(actual, predicted):
  return sqrt(mean_squared_error(actual, predicted))

def train_testsplit(x, y):
  return train_test_split(x, y, test_size=0.3, shuffle=True)

# YOUR CODE HERE
def main():
  print('Hi, I am here 1')
  cpuCount = multiprocessing.cpu_count()
  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  size = comm.Get_size()
  size_of_workers = cpuCount
  feature_chunks, target_chunks = [], []
  print(f"--- Current CPU has {cpuCount} cores ---")
  FILENAME = "/content/PowerPlantData.csv" # File path

  if rank == 0:
    df = loadData(FILENAME)
    df.head()
    exploreData(df)
    df = handleMissingData(df)
    df = scaleData(df)


    # Root worker: Prepare and distribute data
    features, target = featureTargetSplit(df)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

    # Distribute data to all workers (including root worker)
    feature_chunks, target_chunks = dividing_data(X_train, y_train, size_of_workers)

    print(len(feature_chunks))
    print(len(target_chunks))

  feature_scattered = comm.scatter(feature_chunks, root=0)
  target_scattered = comm.scatter(target_chunks, root=0)

  # Calculate the coefficients for each slice of data
  params,X_new = fit(feature_chunks,target_chunks)
  params = comm.gather(params,root=0)

  print('Process {} has gathered data:'.format(rank), params)

# Importing pandas
import pandas as pd
# Importing Numpy
import numpy as np
# Importing sqrt function from the Math
from math import sqrt
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import multiprocessing


main()

Overwriting mpicode.py


In [ ]:
! mpirun --allow-run-as-root --oversubscribe -np 2 python mpicode.py

In [ ]:
! mpirun --allow-run-as-root -np 3 python mpicode.py

#### Exercise 16: Scatter and gather the data (1 point)

Perform the below operations:

- Send slices of the training set(the features data X and the expected target data Y) to every worker including the root worker
    - **Hint:** scatter()
    - use `barrier()` to block workers until all workers in the group reach a Barrier, to scatter from root worker.
- Every worker should get the predicted target Y(yhat) for each slice
- Get the new coefficient of each instance in a slice
    - **Hint:** fit function defined above
- Gather the new coefficient from each worker
    - **Hint:** gather()
    - Take the mean of the gathered coefficients
- Calculate the root mean square error for the test set

To know more about `scatter`, `gather` and `barrier` click [here](https://nyu-cds.github.io/python-mpi/05-collectives/)

In [ ]:
# YOUR CODE HERE


#### Exercise 17: Make a script and execute everything in one place (1 point)

Write a script(.py) file which contains the code of all the above exercises in it so that you can run the code on multiple processes using MPI.

**Hint:**

- magic commands
- put MPI related code under main function
- !mpirun --allow-run-as-root -np 4 python filename.py

In [ ]:
# YOUR CODE HERE for scipt(.py)

In [ ]:
# YOUR CODE HERE for MPI command

#### Exercise 18: Use Sklearn to compare (1 point)

Apply the Linear regression on the given data using sklearn package and compare with the above results

**Hint:**
* Split the data into train and test
* Fit the train data and predict the test data using `sklearn Linear Regression`
* Compare the coefficients and intercept with above estimated coefficients
* calculate loss (RMSE) on test data and predictions and compare

In [ ]:
df.head()

,AmbientTemperature,ExhaustVaccum,AmbientPressure,RelativeHumidity,EnergyOutput
0,-1.517782,-1.065149,-0.407336,1.143885,1.530146
1,0.535228,0.329260,-0.313040,0.061028,-0.504776
2,1.353748,0.204141,-1.028675,-2.150575,-0.914338
3,-0.077992,-0.363223,-1.016888,0.238422,-0.074706
4,-1.053507,-1.073805,0.651804,1.636341,0.589734


In [ ]:
# YOUR CODE HERE

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



regression = LinearRegression()
feature_1, target_1 = featureTargetSplit(df)

X_train, X_test, y_train, y_test = train_test_split(feature_1, target_1, test_size=0.70, random_state=42,shuffle=True)
regression.fit(np.array(X_train),y_train)

# Check the co-efficents from our LinearRegression Class and sklearns
print('SKlearn  Linear Regression Co-efficient is : ',regression.coef_,regression.intercept_)

y_pred = regression.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE Error:", rmse)

SKlearn  Linear Regression Co-efficient is :  [-0.87481436 -0.16774442  0.02299454 -0.13255654] -0.008269650527378537
RMSE Error: 0.26839497882117125


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
